In [0]:
dbutils.fs.mount(
source = "wasbs://toysales-data@toysales.blob.core.windows.net",
mount_point = "/mnt/toysales-data",
extra_configs = {"fs.azure.account.key.toysales.blob.core.windows.net":"c5bwlkfgb8I9R5igDHR2EptCIa437vwloMk31ztQZJChqqW2IBkugpmdQSdzLe870ZD5ur4M8slg+ASt5DELnQ=="})

---------------------------------------------------------------------------
ExecutionError                            Traceback (most recent call last)
File <command-689242910918024>, line 1
----> 1 dbutils.fs.mount(
      2 source = "wasbs://toysales-data@toysales.blob.core.windows.net",
      3 mount_point = "/mnt/toysales-data",
      4 extra_configs = {"fs.azure.account.key.toysales.blob.core.windows.net":"c5bwlkfgb8I9R5igDHR2EptCIa437vwloMk31ztQZJChqqW2IBkugpmdQSdzLe870ZD5ur4M8slg+ASt5DELnQ=="})

File /databricks/python_shell/dbruntime/dbutils.py:362, in DBUtils.FSHandler.prettify_exception_message.<locals>.f_with_exception_handling(*args, **kwargs)
    360 exc.__context__ = None
    361 exc.__cause__ = None
--> 362 raise exc

ExecutionError: An error occurred while calling o429.mount.
: java.rmi.RemoteException: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/toysales-data; nested exception is: 
	java.lang.IllegalArgumentException: requirem

In [0]:
dbutils.fs.ls('/mnt/toysales-data/raw-data')

[FileInfo(path='dbfs:/mnt/toysales-data/raw-data/calendar.csv', name='calendar.csv', size=6930, modificationTime=1703777454000),
 FileInfo(path='dbfs:/mnt/toysales-data/raw-data/inventory.csv', name='inventory.csv', size=14675, modificationTime=1703777466000),
 FileInfo(path='dbfs:/mnt/toysales-data/raw-data/products.csv', name='products.csv', size=1572, modificationTime=1703777479000),
 FileInfo(path='dbfs:/mnt/toysales-data/raw-data/sales.csv', name='sales.csv', size=21783359, modificationTime=1703777495000),
 FileInfo(path='dbfs:/mnt/toysales-data/raw-data/stores.csv', name='stores.csv', size=2999, modificationTime=1703777510000)]

In [0]:
%fs
ls "/mnt/toysales-data/raw-data"

path,name,size,modificationTime
dbfs:/mnt/toysales-data/raw-data/calendar.csv,calendar.csv,6930,1703777454000
dbfs:/mnt/toysales-data/raw-data/inventory.csv,inventory.csv,14675,1703777466000
dbfs:/mnt/toysales-data/raw-data/products.csv,products.csv,1572,1703777479000
dbfs:/mnt/toysales-data/raw-data/sales.csv,sales.csv,21783359,1703777495000
dbfs:/mnt/toysales-data/raw-data/stores.csv,stores.csv,2999,1703777510000


In [0]:
from pyspark.sql.functions import to_date,to_timestamp,col,lit
from pyspark.sql import functions as F, types as T
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DoubleType, BooleanType, DateType, FloatType

In [0]:
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
spark.conf.set("spark.sql.parquet.int96RebaseModeInWrite", "CORRECTED")
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

In [0]:
calendar=spark.read.format("csv").option("header","true").option("header","true").load("/mnt/toysales-data/raw-data/calendar.csv")
inventory=spark.read.format("csv").option("header","true").option("header","true").load("/mnt/toysales-data/raw-data/inventory.csv")
products=spark.read.format("csv").option("header","true").option("header","true").load("/mnt/toysales-data/raw-data/products.csv")
sales=spark.read.format("csv").option("header","true").option("header","true").load("/mnt/toysales-data/raw-data/sales.csv")
stores=spark.read.format("csv").option("header","true").option("header","true").load("/mnt/toysales-data/raw-data/stores.csv")

In [0]:
calendar.show(2)

+--------+
|    Date|
+--------+
|1/1/2022|
|1/2/2022|
+--------+
only showing top 2 rows



In [0]:
calendar.printSchema()

root
 |-- Date: string (nullable = true)



In [0]:
calendar=calendar.withColumn("Date", to_date(col("Date"), "MM/dd/yyyy").cast(DateType()))

In [0]:
calendar.printSchema()

root
 |-- Date: date (nullable = true)



In [0]:
calendar.show(30)

+----------+
|      Date|
+----------+
|2022-01-01|
|2022-01-02|
|2022-01-03|
|2022-01-04|
|2022-01-05|
|2022-01-06|
|2022-01-07|
|2022-01-08|
|2022-01-09|
|2022-01-10|
|2022-01-11|
|2022-01-12|
|2022-01-13|
|2022-01-14|
|2022-01-15|
|2022-01-16|
|2022-01-17|
|2022-01-18|
|2022-01-19|
|2022-01-20|
|2022-01-21|
|2022-01-22|
|2022-01-23|
|2022-01-24|
|2022-01-25|
|2022-01-26|
|2022-01-27|
|2022-01-28|
|2022-01-29|
|2022-01-30|
+----------+
only showing top 30 rows



In [0]:
inventory.show(2)

+--------+----------+-------------+
|Store_ID|Product_ID|Stock_On_Hand|
+--------+----------+-------------+
|       1|         1|           27|
|       1|         2|            0|
+--------+----------+-------------+
only showing top 2 rows



In [0]:
inventory.printSchema()

root
 |-- Store_ID: string (nullable = true)
 |-- Product_ID: string (nullable = true)
 |-- Stock_On_Hand: string (nullable = true)



In [0]:
inventory = inventory.withColumn("Store_ID",col("Store_ID").cast(IntegerType())).withColumn("Product_ID",col("Product_ID").cast(IntegerType())).withColumn("Stock_On_Hand",col("Stock_On_Hand").cast(IntegerType()))

In [0]:
inventory.printSchema()

root
 |-- Store_ID: integer (nullable = true)
 |-- Product_ID: integer (nullable = true)
 |-- Stock_On_Hand: integer (nullable = true)



In [0]:
products.show(2)

+----------+--------------+----------------+------------+-------------+
|Product_ID|  Product_Name|Product_Category|Product_Cost|Product_Price|
+----------+--------------+----------------+------------+-------------+
|         1| Action Figure|            Toys|      $9.99 |      $15.99 |
|         2|Animal Figures|            Toys|      $9.99 |      $12.99 |
+----------+--------------+----------------+------------+-------------+
only showing top 2 rows



In [0]:
products.printSchema()

root
 |-- Product_ID: string (nullable = true)
 |-- Product_Name: string (nullable = true)
 |-- Product_Category: string (nullable = true)
 |-- Product_Cost: string (nullable = true)
 |-- Product_Price: string (nullable = true)



In [0]:
products = products.withColumn('Product_Cost', F.regexp_replace(col('Product_Cost'), '\$', '')).withColumn('Product_Price', F.regexp_replace(col('Product_Price'), '\$', ''))
products = products.withColumn("Product_ID",col("Product_ID").cast(IntegerType())).withColumn("Product_Cost",col("Product_Cost").cast(FloatType())).withColumn("Product_Price",col("Product_Price").cast(FloatType()))

In [0]:
sales.show(2)

+-------+----------+--------+----------+-----+
|Sale_ID|      Date|Store_ID|Product_ID|Units|
+-------+----------+--------+----------+-----+
|      1|2022-01-01|      24|         4|    1|
|      2|2022-01-01|      28|         1|    1|
+-------+----------+--------+----------+-----+
only showing top 2 rows



In [0]:
sales.printSchema()

root
 |-- Sale_ID: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Store_ID: string (nullable = true)
 |-- Product_ID: string (nullable = true)
 |-- Units: string (nullable = true)



In [0]:
sales=sales.withColumn("Sale_ID",col("Sale_ID").cast(IntegerType()))\
    .withColumn("Date", to_date(col("Date"), "yyyy-MM-dd").cast(DateType()))\
        .withColumn("Store_ID",col("Store_ID").cast(IntegerType()))\
            .withColumn("Product_ID",col("Product_ID").cast(IntegerType()))\
                .withColumn("Units",col("Units").cast(IntegerType()))
sales.show(2)

+-------+----------+--------+----------+-----+
|Sale_ID|      Date|Store_ID|Product_ID|Units|
+-------+----------+--------+----------+-----+
|      1|2022-01-01|      24|         4|    1|
|      2|2022-01-01|      28|         1|    1|
+-------+----------+--------+----------+-----+
only showing top 2 rows



In [0]:
stores.show(2)

+--------+--------------------+-----------+--------------+---------------+
|Store_ID|          Store_Name| Store_City|Store_Location|Store_Open_Date|
+--------+--------------------+-----------+--------------+---------------+
|       1|Maven Toys Guadal...|Guadalajara|   Residential|     1992-09-18|
|       2|Maven Toys Monter...|  Monterrey|   Residential|     1995-04-27|
+--------+--------------------+-----------+--------------+---------------+
only showing top 2 rows



In [0]:
stores.printSchema()

root
 |-- Store_ID: string (nullable = true)
 |-- Store_Name: string (nullable = true)
 |-- Store_City: string (nullable = true)
 |-- Store_Location: string (nullable = true)
 |-- Store_Open_Date: string (nullable = true)



In [0]:
stores=stores.withColumn("Store_ID",col("Store_ID").cast(IntegerType()))\
    .withColumn("Store_Open_Date", to_date(col("Store_Open_Date"), "yyyy-MM-dd").cast(DateType()))
stores.show(2)

+--------+--------------------+-----------+--------------+---------------+
|Store_ID|          Store_Name| Store_City|Store_Location|Store_Open_Date|
+--------+--------------------+-----------+--------------+---------------+
|       1|Maven Toys Guadal...|Guadalajara|   Residential|     1992-09-18|
|       2|Maven Toys Monter...|  Monterrey|   Residential|     1995-04-27|
+--------+--------------------+-----------+--------------+---------------+
only showing top 2 rows



In [0]:
# calculate total cost
total_cost=products.join(sales).select("Product_Name","Units","Product_Price", (products.Product_Price * sales.Units).alias("Total_Cost"))
total_cost.show()

+--------------------+-----+-------------+----------+
|        Product_Name|Units|Product_Price|Total_Cost|
+--------------------+-----+-------------+----------+
|       Action Figure|    1|        15.99|     15.99|
|      Animal Figures|    1|        12.99|     12.99|
|     Barrel O' Slime|    1|         3.99|      3.99|
|    Chutes & Ladders|    1|        12.99|     12.99|
|    Classic Dominoes|    1|         9.99|      9.99|
|           Colorbuds|    1|        14.99|     14.99|
|            Dart Gun|    1|        15.99|     15.99|
|       Deck Of Cards|    1|         6.99|      6.99|
|            Dino Egg|    1|        10.99|     10.99|
|    Dinosaur Figures|    1|        14.99|     14.99|
|       Etch A Sketch|    1|        20.99|     20.99|
|  Foam Disk Launcher|    1|        11.99|     11.99|
|    Gamer Headphones|    2|        20.99|     41.98|
|       Glass Marbles|    1|        10.99|     10.99|
|   Hot Wheels 5-Pack|    1|         5.99|      5.99|
|               Jenga|    1|

In [0]:
# Top 10 products sold
top_products_sold=total_cost.groupBy("Product_Name").agg({'Total_Cost': 'max'}).withColumnRenamed("max(Total_Cost)", "Revenue")
top_products_sold=top_products_sold.orderBy("Revenue", ascending=False).select("Product_Name","Revenue").show(10)

+--------------------+---------+
|        Product_Name|  Revenue|
+--------------------+---------+
|         Lego Bricks|1199.7001|
|           Toy Robot|    779.7|
|     PlayDoh Playset|    749.7|
|Mini Basketball Hoop|    749.7|
|    Gamer Headphones|    629.7|
|       Etch A Sketch|    629.7|
|        Rubik's Cube|    599.7|
|     Kids Makeup Kit|    599.7|
|            Nerf Gun|    599.7|
|            Monopoly|    599.7|
+--------------------+---------+
only showing top 10 rows



In [0]:
# Total stores by city
total_stores=stores.groupBy("Store_Name","Store_City").sum()
total_stores=total_stores.orderBy("sum(Store_ID)", ascending=False)
total_stores=total_stores.withColumnRenamed("sum(Store_ID)","Total").show(10)
# filter by city
stores.filter(stores["Store_City"]=="Guanajuato").show()

+--------------------+----------------+-----+
|          Store_Name|      Store_City|Total|
+--------------------+----------------+-----+
|Maven Toys Guanaj...|      Guanajuato|   50|
|Maven Toys Culiac...|        Culiacan|   49|
|Maven Toys Saltil...|        Saltillo|   48|
|Maven Toys Monter...|       Monterrey|   47|
|Maven Toys Guadal...|     Guadalajara|   46|
|Maven Toys Ciudad...|Cuidad de Mexico|   45|
| Maven Toys Puebla 3|          Puebla|   44|
|Maven Toys Durango 1|         Durango|   43|
|Maven Toys Hermos...|      Hermosillo|   42|
|Maven Toys Hermos...|      Hermosillo|   41|
+--------------------+----------------+-----+
only showing top 10 rows

+--------+--------------------+----------+--------------+---------------+
|Store_ID|          Store_Name|Store_City|Store_Location|Store_Open_Date|
+--------+--------------------+----------+--------------+---------------+
|      14|Maven Toys Guanaj...|Guanajuato|      Downtown|     2007-01-31|
|      22|Maven Toys Guanaj...|Gua

In [0]:
calendar.repartition(1).write.mode("overwrite").option("header",'true').csv("/mnt/toysales-data/transformed-data/calendar")
inventory.repartition(1).write.mode("overwrite").option("header",'true').csv("/mnt/toysales-data/transformed-data/inventory")
products.repartition(1).write.mode("overwrite").option("header",'true').csv("/mnt/toysales-data/transformed-data/products")
sales.repartition(1).write.mode("overwrite").option("header",'true').csv("/mnt/toysales-data/transformed-data/sales")
stores.repartition(1).write.mode("overwrite").option("header",'true').csv("/mnt/toysales-data/transformed-data/stores")